In [1]:
import sqlalchemy
from sqlalchemy import create_engine, Column, Integer, String, Sequence, ForeignKey, Time
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship

In [9]:
engine = create_engine('sqlite:///iProject.db', echo=True)
Base = declarative_base()

In [10]:
class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, Sequence('player_id_seq'), primary_key=True)
    rank = Column(String(1))
    cash = Column(Integer)
    coin = Column(Integer)
    heroes_id = Column(Integer, ForeignKey('heroes.id'))
    heroes = relationship("Hero")
    skins_id = Column(Integer, ForeignKey('skins.id'))
    skins = relationship("Skin")
    match_history_id = Column(Integer, ForeignKey('matches.id'))
    match_history = relationship("Match")
    friends_id = Column(Integer, ForeignKey('users.id'))
    friends = relationship("User")
    
    def __init__(self, name, server):
        self.name = name
        self.server = server
        server.online_users.append(self)
    def __repr__(self):
        return 'User: %s\
                rank: %s\
                cash: %d\
                coin: %d' % (self.name,
                            self.rank,
                            self.cash,
                            self.coin)    # Can't loop Heros name
    def rankUP(self):
        pass 
    def rankDOWN(self):
        pass 
    def insert_cash(self, m):
        self.cash = self.cash + m
        self.server.wallet = self.server.wallet + m
    def buyHero(self, c, hero):
        if(c == 'cash'):
            if(self.cash >= hero.cash_price):
                self.cash = self.cash - hero.cash_price
                self.heroes.append(hero)
            else:
                print("Your cash not enough")
        else:
            if(self.coin >= hero.coin_price):
                self.coin = self.coin - hero.coin_price
                self.heroes.append(hero)
            else:
                print("Your coin not enough")

                
class Hero(Base):
    __tablename__ = 'heroes'
    id = Column(Integer, Sequence('hero_id_seq'), primary_key=True)
    healthPoint = Column(Integer)
    manaPoint = Column(Integer)
    damage = Column(Integer)
    cash_price = Column(Integer)
    coin_price = Column(Integer)
    skins_id = Column(Integer, ForeignKey('skins.id'))
    skins = relationship("Skin")
    skills_id = Column(Integer, ForeignKey('skills.id'))
    skills = relationship("Skill")
    ingame_id = Column(Integer)
    __mapper_args__ = {
        'polymorphic_on' : ingame_id
    }
    
    def __init__(self, name, skin=None, cash_price=0, coin_price=0, hp=300, damage=20):
        self.name = name
#         self.skins.append(skin)
        self.cash_price = cash_price
        self.coin_price = coin_price
        self.healthPoint = hp
        self.damage = damage
#     def buySkin(self, buyer, skin):
#         if(skin in self.skins):
#             if(self in buyer.heros):
#                 if(buyer.cash >= skin.price):
#                     buyer.cash = buyer.cash - skin.price
#                     self.skins.append(skin)
#                 else:
#                     print("cash not enough")
#             else:
#                 print("Buy Hero First")
#         else:
#             print("%s is not skin of %s"% (skin.pic, self.name))
    
    
class Skin(Base):
    __tablename__ = 'skins'
    id = Column(Integer, Sequence('skin_id_seq'), primary_key=True)
    pic = Column(String)
    price = Column(Integer)
    
    def __init__(self, pic, price=0):
        self.pic = pic
        self.price = price

        
class Skill(Base):
    __tablename__ = 'skills'
    id = Column(Integer, Sequence('skill_id_seq'), primary_key=True)
    str_damage = Column(Integer)
    magic_damage = Column(Integer)
    cooldown = Column(Integer)
    
    def __init__(self, str_damage, magic_damage, cooldown):
        self.str_damage = str_damage
        self.magic_damage = magic_damage
        self.cooldown = cooldown

        
class InGameHero(Hero):
    __tablename__ = 'ingamehero'
    id = Column(Integer, Sequence('ingamehero_id_seq'), ForeignKey('heroes.id'), primary_key=True)
    level = Column(Integer)
    money = Column(Integer)
    items_id = Column(Integer, ForeignKey('itemsingame.id'))
    items = relationship("InGameItem")
    useable_skills_id = Column(Integer, ForeignKey('skills.id'))
    useable_skills = relationship("Skill")
    total_dmg = Column(Integer)
    kill = Column(Integer)
    die = Column(Integer)
    assist = Column(Integer)
    team = Column(String)
    __mapper_args__ = {
        'polymorphic_identity' : 'ingamehero'
    }
    
    def __init__(self, user,team , skin=None):
        self.user = user
        self.skin = skin
        self.team = team
#     def buyItem(self, item):
#         if(len(self.item) < 6):
#             if(self.money >= item.buyprice):
#                 self.item.append(item)
#                 self.money = self.money - item.buyprice
#             else:
#                 print("not enough money")
#         else:
#             print("Full ITEMS")
#     def sellItem(self, idx):
#         self.money = self.money + self.item[idx].sellprice
#         self.item.remove(idx)
    
    
class InGameItem(Base):
    __tablename__ = 'itemsingame'
    id = Column(Integer, Sequence('itemingame_id_seq'),primary_key=True)
    buyprice = Column(Integer)
    sellprice = Column(Integer)
    prop = Column(String)
    component_id = Column(Integer, ForeignKey('itemsingame.id'))
    components = relationship("InGameItem")

    def __init__(self, bp, sp, ppt, *arg):
        self.buyprice = bp
        self.sellprice = sp
        self.prop = ppt
        self.components.append(arg)
    
    
class Match(Base):
    __tablename__ = 'matches'
    id = Column(Integer, Sequence('match_id_seq'), primary_key=True)
    radiant_team_id = Column(Integer, ForeignKey('users.id'))
    radiant_team = relationship("User")
    dire_team_id = Column(Integer, ForeignKey('users.id'))
    dire_team = relationship("User")
    win_team = Column(String)
    time = Column(Time(timezone=False))
    
#     def __init__(r1, r2, r3, r4, r5, b1, b2, b3, b4, b5):
#         self.radiant_team.append(r1, r2, r3, r4, r5)
#         self.dire_team.append(b1, b2, b3, b4, b5)
#         raise except('import all in game items')
    def select_hero():
        raise Exception('Select heroes and create in game heroes')
    def start():
        raise Exception('Start game... HOW???')
#     def dispense(self, coin, *heroes):
#         for hero in heroes:
#             hero.user.coin = hero.user.coin + coin
#     def endgame():
#         if win_team == 'radiant':
#             dispense(10000, radiant_team)
#             dispense(5000, dire_team)
#         else:
#             dispense(5000, radiant_team)
#             dispense(10000, dire_team)

    
class Item(Base):
    __tablename__ = 'items'
    id = Column(Integer, Sequence('item_id_seq'), primary_key=True)
    cash_price = Column(Integer)
    coin_price = Column(Integer)
    prop = Column(String)
    
    def __init__(self, prop, cash_price = 0, coin_price = 0):
        self.prop = prop
        self.cash_price = cash_price
        self.coin_price = coin_price
    
    
class Shop(Base):
    __tablename__ = 'Shops'
    id = Column(Integer, Sequence('shop_id_seq'), primary_key=True)
    items_id = Column(Integer, ForeignKey('items.id'))
    items = relationship("Item")
    skins_id = Column(Integer, ForeignKey('skins.id'))
    skins = relationship("Skin")
    heroes_id = Column(Integer, ForeignKey('heroes.id'))
    items = relationship("Hero")

    
class Server(Base):
    __tablename__ = 'servers'
    id = Column(Integer, Sequence('server_id_seq'), primary_key=True)
    server_name = Column(String)
    max_user = Column(Integer)
    online_users_id = Column(Integer, ForeignKey('users.id'))
    online_users = relationship("User")
    wallet = Column(Integer)
    unit = Column(String(3))    # unit of money
    
    def __repr__(self):
        return '%s Server[max_user = %d, wallet = %d %s]'%(self.server_name, self.max_user, self.wallet, self.unit)

In [11]:
User.__table__

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False, default=Sequence('player_id_seq', metadata=MetaData(bind=None))), Column('rank', String(length=1), table=<users>), Column('cash', Integer(), table=<users>), Column('coin', Integer(), table=<users>), Column('heroes_id', Integer(), ForeignKey('heroes.id'), table=<users>), Column('skins_id', Integer(), ForeignKey('skins.id'), table=<users>), Column('match_history_id', Integer(), ForeignKey('matches.id'), table=<users>), Column('friends_id', Integer(), ForeignKey('users.id'), table=<users>), schema=None)

In [12]:
Base.metadata.create_all(engine)

2020-07-06 12:00:16,145 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-07-06 12:00:16,150 INFO sqlalchemy.engine.base.Engine ()
2020-07-06 12:00:16,150 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-07-06 12:00:16,150 INFO sqlalchemy.engine.base.Engine ()
2020-07-06 12:00:16,160 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("users")
2020-07-06 12:00:16,160 INFO sqlalchemy.engine.base.Engine ()
2020-07-06 12:00:16,160 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("users")
2020-07-06 12:00:16,170 INFO sqlalchemy.engine.base.Engine ()
2020-07-06 12:00:16,170 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("heroes")
2020-07-06 12:00:16,170 INFO sqlalchemy.engine.base.Engine ()
2020-07-06 12:00:16,180 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("heroes")
2020-07-06 12:00:16,180 INFO sqlalchemy.engine.base.Engine ()
2020-07-06 12:00:16,180 

In [13]:
thai_server = Server(server_name='Thai', max_user=10000, wallet=0, unit='THB')
us_server = Server(server_name='USA',max_user=15000,wallet=0,unit='USD')
eu_server = Server(server_name='Europe',max_user=12000,wallet=0,unit='EUR')

AmbiguousForeignKeysError: Could not determine join condition between parent/child tables on relationship User.match_history - there are multiple foreign key paths linking the tables.  Specify the 'foreign_keys' argument, providing a list of those columns which should be counted as containing a foreign key reference to the parent table.

In [7]:
Session = sessionmaker(bind=engine)
session = Session()
session

In [8]:
for serve in [thai_server, us_server, eu_server]:
    session.add(serve)

In [11]:
all_server = session.query(Server)

In [10]:
session.new

IdentitySet([])

In [44]:
thai_server

Thai Server[max_user = 10000, wallet = 0 THB]

In [12]:
session.commit()

2020-07-05 17:27:11,936 INFO sqlalchemy.engine.base.Engine COMMIT


In [13]:
session.close()

In [14]:
all_server